In [1]:
import numpy as np

In [2]:
#GLRAM core code
def update_L(A_list, R, k):
    """Update L given R."""
    ML = np.sum([Ai @ R @ R.T @ Ai.T for Ai in A_list], axis=0)
    eigenvalues, eigenvectors = np.linalg.eig(ML)
    # 取最大的k个特征值对应的特征向量
    idx = np.argsort(eigenvalues)[-k:][::-1]
    return eigenvectors[:, idx]

def update_R(A_list, L, k):
    """Update R given L."""
    MR = np.sum([Ai.T @ L @ L.T @ Ai for Ai in A_list], axis=0)
    eigenvalues, eigenvectors = np.linalg.eig(MR)
    # 取最大的k个特征值对应的特征向量
    idx = np.argsort(eigenvalues)[-k:][::-1]
    return eigenvectors[:, idx]

def als_algorithm(A_list, init_L, init_R, k1, k2, iterations=100):
    """
    Alternating Least Squares algorithm for optimizing L and R.
    
    Parameters:
    A_list (list of np.ndarray): 数据矩阵列表。
    init_L, init_R (np.ndarray): L和R的初始估计，形状分别为(l, k1)和(k2, c)。
    k1, k2 (int): L和R的目标秩。
    iterations (int): 迭代次数。
    
    Returns:
    L, R (np.ndarray): 最终的L和R矩阵。
    """
    L = init_L
    R = init_R
    
    for _ in range(iterations):
        # 更新L
        L = update_L(A_list, R, k1)
        # 更新R，注意保持R的列正交性可能需要额外步骤，这里简化处理
        R = update_R(A_list, L, k2)
        
        # 在实际应用中，可能需要额外的正交化步骤来确保L和R的正交性
        # 但这里为了简化，我们直接使用了特征向量，它们自然正交
        
    return L, R


In [ ]:
# the RMSRE of GlRAM 
def Frobenius_norm (matrix):
    a = np.linalg.norm(matrix,'fro')
    return a
def cal_GLRAM_RMSRE (matrix_list,L,R):
    sum=0
    num = 0
    for ai in matrix_list:
        temp = np.dot(np.dot(L,L.T),np.dot(ai,np.dot(R,R.T)))
        num += 1
        error = ai - temp
        sum += Frobenius_norm (error)**2
    rmsre =   np.sqrt( sum/num)
    return rmsre